In [1]:
import andi
import numpy as np
import csv as csv

from sklearn.metrics import mean_absolute_error
import tensorflow as tf



import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras import losses, metrics

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import LSTM

from keras.layers import Dropout
from keras.layers import Bidirectional
#from keras.models import load_model
from tensorflow.keras.models import load_model
import os
from matplotlib import pyplot as plt
#from data_split import data_split
from utils import data_norm, data_reshape, many_net_uhd

Using TensorFlow backend.


In [ ]:
os.getcwd()

Importing the data

In [3]:
import csv
trajs_from_files = csv.reader(open('challenge_for_scoring/task1.txt','r'), delimiter=';', 
                                        lineterminator='\n',quoting=csv.QUOTE_NONNUMERIC)
validation = [[],[],[]]
for trajs in enumerate(trajs_from_files):
    validation[int(trajs[1][0])-1].append(trajs[1][1:])

Generating trajectories to test the data. 3 trajectories for each dimension of inference and classification tasks

In [26]:
AD = andi.andi_datasets()
i=200

X1, Y1, X2, Y2, X3, Y3 = AD.andi_dataset(N = 3, tasks = [1,2], dimensions = [1,2,3],
                                        min_T = i, max_T = i+1, )

Creating a dataset for task(s) [1, 2] and dimension(s) [1, 2, 3].
Generating dataset for dimension 1.
Generating dataset for dimension 2.
Generating dataset for dimension 3.


# Inference of 1d trajectories

The output of the inference networks is the inferred anomalous exponent for each trajectory. Trajectories must be of equal length

Importing the networks used for inference in 1d

In [2]:
centers_inf_1d = [25, 50, 65, 75, 125, 165, 225, 
                  325, 425, 525, 625, 725, 825, 925]
meta_model_inf_1d = []
for i in centers_inf_1d: 
    m = load_model('nets/inference_nets/1d/inference_1D_'+str(i)+'.h5')
    
    meta_model_inf_1d.append(m)
    

Using the net trained on trajectories of length 165

In [27]:
#choosing the net
net = meta_model_inf_1d[5]
#finding out the block size used by the chosen net
bs = net.layers[0].input_shape[-1]

#normalizing the data
data = data_norm(X1[0],dim=1,task=1)


#reshaping the data

data_rs = data_reshape(data,bs=bs,dim=1)
#prediction on trajectories of length 200 using a net trained on traj of length 165
pred_200_u165 = net.predict(data_rs)
print('predicted exponents', pred_200_u165.flatten())
print('ground truth', Y1[0])

predicted exponents [1.845595   1.6862895  0.92254865]
ground truth [1.8, 1.9, 1.0]


Using the net trained on trajectories of length 225

In [29]:
#choosing the net
net = meta_model_inf_1d[6]
#finding out the block size used by the chosen net
bs = net.layers[0].input_shape[-1]

#normalizing the data
data = data_norm(X1[0],dim=1,task=1)


#reshaping the data

data_rs = data_reshape(data,bs=bs,dim=1)
#prediction on trajectories of length 200 using a net trained on traj of length 225
pred_200_u225 = net.predict(data_rs)
print('predicted exponents', pred_200_u225.flatten())
print('ground truth', Y1[0])

predicted exponents [1.8311543  1.6214162  0.86620164]
ground truth [1.8, 1.9, 1.0]


Using the combination of nearest nets, which in this case is 165 and 225 

In [28]:
pred_200_comb = many_net_uhd(nets = meta_model_inf_1d, traj_set = X1[0], centers = centers_inf_1d ,dim = 1, task =1)
print('predicted exponents',pred_200_comb)
print('ground truth', Y1[0])

predicted exponents [1.8371712  1.6484468  0.88967955]
ground truth [1.8, 1.9, 1.0]


# Classification in 2d

The output of the classiciation networks is an array giving the probability that the trajectory belongs to a model class. The classes, as in AnDi are [attm,ctrw,fbm,lw,sbm]

In [30]:
centers_class_2d = [25, 65, 125, 225, 425]
meta_model_class_2d = []
for i in centers_class_2d: 
    m = load_model('nets/classification_nets/2d/classification_2D_'+str(i)+'.h5')
    
    meta_model_class_2d.append(m)
    

In [31]:
#choosing the net
net = meta_model_class_2d[2]
#finding out the block size used by the chosen net
bs = net.layers[0].input_shape[-1]

#normalizing the data
data = data_norm(X2[1],dim=2,task=2)


#reshaping the data

data_rs = data_reshape(data,bs=bs,dim=2)
#prediction on trajectories of length 200 using a net trained on traj of length 165
cla_200_u125 = net.predict(data_rs)
print("probability of each model class",'\n',cla_200_u125)
print("predicte most likely model")
models = ['attm', 'ctrw', 'fbm', 'lw', 'sbm'] 
for i in range(len(data)):
    print(models[np.argmax(cla_200_u125[i])])
print('Ground truth')    
for i in range(len(data)):
    print(models[int(Y2[1][i])])

probability of each model class 
 [[9.9778748e-01 5.4923461e-05 1.1240288e-05 9.2051545e-05 2.0543423e-03]
 [8.6530723e-05 9.5793021e-01 4.1861311e-02 1.1974435e-04 2.1988903e-06]
 [1.1371747e-03 5.9122603e-06 8.0086995e-04 1.2150083e-06 9.9805486e-01]]
predicte most likely model
attm
ctrw
sbm
Ground truth
attm
ctrw
sbm


Combining the nets

In [32]:
cla_200_comb = many_net_uhd(nets = meta_model_class_2d, traj_set = X2[1], centers = centers_class_2d ,dim = 2, task =2)
cla_200_comb = cla_200_comb.reshape(-1,5)
print("probability of each model class",'\n',cla_200_comb)
print("most likely model")
models = ['attm', 'ctrw', 'fbm', 'lw', 'sbm'] 
for i in range(len(data)):
    print(models[np.argmax(cla_200_comb[i])])
print('Ground truth')    
for i in range(len(data)):
    print(models[int(Y2[1][i])])    

probability of each model class 
 [[9.9878442e-01 5.5976030e-05 2.5782732e-05 2.0775158e-04 9.2605810e-04]
 [2.0063552e-04 8.5220039e-01 1.4750800e-01 7.0587907e-05 2.0408525e-05]
 [1.5283580e-03 5.0323297e-06 5.1123847e-04 9.1584417e-07 9.9795437e-01]]
most likely model
attm
ctrw
sbm
Ground truth
attm
ctrw
sbm
